In [15]:
import pandas as pd
import numpy as np

import os

def load_hotel_reserve():
  customer_tb = pd.read_csv('./data/customer.csv')
  hotel_tb = pd.read_csv('./data/hotel.csv')
  reserve_tb = pd.read_csv('./data/reserve.csv')
  return customer_tb, hotel_tb, reserve_tb
 

def load_holiday_mst():
  holiday_tb = pd.read_csv('./data/holiday_mst.csv',
                           index_col=False)
  return holiday_tb


def load_production():
  production_tb = pd.read_csv('./data/production.csv')
  return production_tb


def load_production_missing_num():
  production_tb = pd.read_csv('./data/production_missing_num.csv')
  return production_tb


def load_production_missing_category():
  production_tb = pd.read_csv('./data/production_missing_category.csv')
  return production_tb


def load_monthly_index():
  monthly_index_tb = \
    pd.read_csv('./data/monthly_index.csv')
  return monthly_index_tb


def load_meros_txt():
  with open('./data/txt/meros.txt', 'r') as f:
    meros = f.read()
    f.close()
  return meros


In [16]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

#### 4. 위치정보 데이터 전처리 기법
- 지역 기준 좌표계 변환

    -> 2010년 1월부터 공공측량 성과물에 세계 측지계 사용이 의무화

    -> 따라서 한국 측지계 데이터는 세계 측지계로 변환하여 사용하여야 함

In [17]:
# 기존의 한국 축지계
customer_tb[['home_longitude', 'home_latitude']]

,home_longitude,home_latitude
0,136.512347,35.092193
1,139.410551,35.325076
2,136.511179,35.120543
3,141.240314,43.034868
4,136.523797,35.102661
...,...,...
995,135.373787,34.465648
996,139.413754,35.345372
997,141.272126,43.062267
998,140.464198,38.172800


In [18]:
# 고객 테이블에서 집의 위도와 경도를 도 단위로 변경한 후 세계 측지계로 변환
import pyproj

def convert_to_continuous(x):
    x_min = (x * 100 - int(x * 100)) * 100
    x_sec = (x - int(x) - x_min / 10000) * 100
    return int(x) + x_sec / 60 + x_min / 60 / 60

customer_tb['home_latitude'] = customer_tb['home_latitude']\
    .apply(lambda x: convert_to_continuous(x))
customer_tb['home_longitude'] = customer_tb['home_longitude']\
    .apply(lambda x: convert_to_continuous(x))

epsg_world = pyproj.Proj('+init=EPSG:4326') # 세계 축지계

epsg_korea = pyproj.Proj('+init=EPSG:4301') # 한국 축지계

home_position = customer_tb[['home_longitude', 'home_latitude']]\
    .apply(lambda x:
           pyproj.transform(epsg_korea, epsg_world, x[0], x[1]), axis=1) # 한국 축지계를 세계 축지계로 변환

customer_tb['home_longitude'] = [x[0] for x in home_position]
customer_tb['home_latitude'] = [x[1] for x in home_position] # 세계 축지계로 갱신

customer_tb[['home_longitude', 'home_latitude']]

c:\Users\EL087\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\EL087\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\EL087\AppData\Local\Temp\ipykernel_20672\2868518265.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future vers

,home_longitude,home_latitude
0,136.853555,35.159315
1,139.681642,35.550685
2,136.850310,35.204727
3,141.397126,43.065946
4,136.874247,35.177280
...,...,...
995,135.624343,34.785592
996,139.690537,35.584837
997,141.452152,43.108717
998,140.774893,38.294081


- 두 지점 간 거리 및 방향 계산

    -> 위도와 경도를 이용하여 두 지점 간의 거리와 방향을 계산

    -> 200km 이내일 때 휴베니 방식, 그 이상에서 정확도를 요구할 때 빈센티 또는 하버사인 방식 사용

In [20]:
# python으로 위도 경도 위치정보를 다루기위한 라이브러리 로드
import math
import pyproj

# 거리를 계산하기 위한 라이브러리 로드
#from geopy.distance import great_circle, vincenty
from geopy.distance import great_circle

# ・・・측지계 변화 코드는 생략・・・

# 예약 테아블에 고객 테이블과 호텔 테이블을 결합
reserve_tb = \
  pd.merge(reserve_tb, customer_tb, on='customer_id', how='inner')
reserve_tb = pd.merge(reserve_tb, hotel_tb, on='hotel_id', how='inner')

# 집과 호텔의 위도 경도 정보를 획득
home_and_hotel_points = reserve_tb \
  .loc[:, ['home_longitude', 'home_latitude',
           'hotel_longitude', 'hotel_latitude']]

# 적도 반경을 WGS84기준으로 설정
g = pyproj.Geod(ellps='WGS84')

# 방위각 반방위각, Vincenty식을 이용한 거리 계산
home_to_hotel = home_and_hotel_points \
  .apply(lambda x: g.inv(x[0], x[1], x[2], x[3]), axis=1)

# 방위각을 구함
[x[0] for x in home_to_hotel]

# Vincenty식을 이용한 거리를 구함
[x[2] for x in home_to_hotel]

# Haversine식을 이용한 거리 계산
home_and_hotel_points.apply(
  lambda x: great_circle((x[1], x[0]), (x[3], x[2])).meters, axis=1)

# Vincenty식을 이용한 거리 계산
#home_and_hotel_points.apply(
#  lambda x: vincenty((x[1], x[0]), (x[3], x[2])).meters, axis=1)


# Hubeny식의 함수 정의
def hubeny(lon1, lat1, lon2, lat2, a=6378137, b=6356752.314245):
    e2 = (a ** 2 - b ** 2) / a ** 2
    (lon1, lat1, lon2, lat2) = \
      [x * (2 * math.pi) / 360 for x in (lon1, lat1, lon2, lat2)]
    w = 1 - e2 * math.sin((lat1 + lat2) / 2) ** 2
    c2 = math.cos((lat1 + lat2) / 2) ** 2
    return math.sqrt((b ** 2 / w ** 3) * (lat1 - lat2) ** 2 +
                     (a ** 2 / w) * c2 * (lon1 - lon2) ** 2)

# Hubeny식을 이용한 거리 계산
home_and_hotel_points \
  .apply(lambda x: hubeny(x[0], x[1], x[2], x[3]), axis=1)


C:\Users\EL087\AppData\Local\Temp\ipykernel_20672\781485203.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  .apply(lambda x: g.inv(x[0], x[1], x[2], x[3]), axis=1)
C:\Users\EL087\AppData\Local\Temp\ipykernel_20672\781485203.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: great_circle((x[1], x[0]), (x[3], x[2])).meters, axis=1)
C:\Users\EL087\AppData\Local\Temp\ipykernel_20672\781485203.py:55: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by positi

0       262390.186582
1       263567.677155
2       597948.485992
3       498654.036378
4       291511.077110
            ...      
4025    534644.527104
4026      8293.928977
4027    533501.714676
4028    313055.146941
4029     21691.391897
Length: 4030, dtype: float64